In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.io as sio

from PIL import Image
from skimage.util import img_as_float
from skimage.color import rgb2gray
from skimage.io import imread, imsave
from scipy.io import loadmat
from tqdm import tqdm

import cv2
import urllib.request
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
data_dir = "/content/gdrive/MyDrive/CV/termProject/project_data"
pred_dir = "/content/gdrive/MyDrive/CV/termProject/predictions"

# Checking if our specified directory exists
os.path.exists(data_dir)

# Train data 불러오기

In [4]:
# train data 불러오는 class

class BoundaryDataset(object):
    """
    Project boundary dataset wrapper

    Given the path to the root of the dataset, this class provides
    methods for loading images and ground truths.

    Attributes:

    root_dir - the root path of the dataset
    data_path - the path of the data
    directory within the root
    sample_names - a list of names of images
    """
    def __init__(self, root_dir='.', split='train'):
        """
        Constructor

        :param root_dir: the path to the root of the custom dataset
        :param split: 'train' or 'test'
        """
        self.root_dir = root_dir
        self.data_path = os.path.join(root_dir, split)
        self.sample_names = self._sample_names(self.data_path)

    def __len__(self):
        """
        Get the number of samples in the dataset
        :return: the number of samples
        """
        return len(self.sample_names)

    @staticmethod
    def _sample_names(directory):
        names = []
        files = os.listdir(directory)
        for fn in files:
            name, ext = os.path.splitext(fn)
            if ext.lower() == '.jpg':
                names.append(name)
        return names

    def read_image(self, name):
        """
        Load the image identified by the sample name
        :param name: the sample name
        :return: a (H,W,3) array containing the image
        """
        path = os.path.join(self.data_path, f"{name}.jpg")
        return imread(path)

    def load_boundaries(self, name):
        """
        Load the boundaries identified by the sample name
        :param name: the sample name
        :return: a list of (H,W) arrays, each of which contains a boundary ground truth
        """
        boundary_path = os.path.join(self.data_path, f"{name}.npy")
        if os.path.exists(boundary_path):
            boundaries = np.load(boundary_path, allow_pickle=True)
            return list(boundaries)
        return []

In [ ]:
num_images = 10
split = 'train'

# Load the dataset using BoundaryDataset class
dataset = BoundaryDataset(data_dir, split=split)

print(f"Data size: {len(dataset)}")

# ground truth 함께 확인

```
# of test image : [:20] -> F1 (Recall / Precision)
Overall Evaluation Metrics Using a Single Best Threshold for All Images
```

- ground truth[0] -> 0.917886 (0.848444 / 0.999709)
- ground truth[1] -> 0.885948 (0.795358 / 0.999828)


In [6]:
random.seed(100)
selected_sample_names = random.sample(dataset.sample_names, min(num_images, len(dataset.sample_names)))
selected_sample_names = sorted(dataset.sample_names)[100:110]

In [ ]:

data_dic = {}
# Plot multiple images and their boundaries
for selected_sample_name in dataset.sample_names:

    data_dic[selected_sample_name] = ''

    print(f"Processing image: {split}/{selected_sample_name}")
    image = dataset.read_image(selected_sample_name)
    boundaries = dataset.load_boundaries(selected_sample_name)

    fig, ax = plt.subplots(1, len(boundaries) + 1, figsize=(6 * (len(boundaries) + 1), 6))

    ax[0].imshow(image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    for i in range(len(boundaries)):
        ax[i + 1].imshow(boundaries[i], cmap='gray')
        ax[i + 1].set_title(f"Boundary {i + 1}")
        ax[i + 1].axis("off")

    plt.tight_layout()
    plt.show()

# ⭐️gaussian blur + median blur


```
[ (gblur kernelsize), mblur kernelsize, mblur의 blend 비율, (canny thre) ]

# of test image : [:20] -> F1 (Recall / Precision)
Overall Evaluation Metrics Using a Single Best Threshold for All Images
-> ( Threshold  Recall     Precision  F1-Score)

```

- test02_0_2blur_all.npy : [ ?? ] -> 0.591017  
- test02_1_2blur_all.npy : [ ?? ] -> 0.588647 (0.611680 / 0.567285)
- test02_2_2blur_all.npy : [ (11,11), 21, 0.45, (3,80) ] -> 0.588586 (0.611739 / 0.567654)

    [200] -> 0.549023 (0.610456 / 0.498824)
- test02_3_2blur_all.npy : [ (11,11), 17, 0.5, (3,80) ] -> 0.599114 (0.622844 / 0.577126)

    [200] -> 0.554463 (0.613340 / 0.505900)
- test02_4_2blur_all.npy : [ (11,11), 17, 0.45, (5,95) ] -> 0.600955 (0.582339 / 0.620801) : 바운더리를 좀 더 찾아내야 하는 상황

    [:100]으로 돌리니까 갑자기 (re0.853052  pre0.334845  F0.480918)  이러는데 하....-> precision이개낮음

    [200] ->  0.477919  (0.850314 /0.332362)
- test02_5_2blur_all.npy : [ (11,11), 15, 0.45, (0,110) ] -> 0.588665 (0.538169 / 0.649618): thre2를 높게 줘서 확실한 엣지는 엣지로 박아두고, 나머지 픽셀은 이어져잇으면 엣지로 취급하는 방법...

    [200] -> 0.547980 (0.538143  / 0.558183 )
- test02_6_2blur_all.npy : [ (11,11), 15, 0.45, (250,253) ] -> 0.158472 (0.087858 / 0.807389)  : 그냥 극단적으로 엣지를 없애서.. precision을 높였을 때 recall이 어케될지,, 오 완전 구려지넹.. F1 score라서 그냥 둘 다 높아야 한다...
- test02_7_2blur_all.npy : [ (11,11), 17, 0.5, (3,70) ] -> [200] 0.548656 (0.657777 / 0.470588)   : 미디안 좀 올려보고 싶음
- test02_8_2blur_all.npy : [ (11,11), 17, 0.6, (3,70) ] -> [200] 0.551136 (0.621892 / 0.494835)   
- test02_9_2blur_all.npy : [ (9,9), 17, 0.6, (3,70) ] -> [200] 0.539858 (0.685454 / 0.445278)
- test02_10_2blur_all.npy : [ (9,9), 17, 0.65, (15,90) -> [200] 0.549334 (0.572947 / 0.527590) :눈으로 봐서는 이제 모르겠다 하.. 얇은 엣지를 못찾고, 안 찾아도 될 디테일은 찾고..
- test02_11_2blur_all.npy : [ (9,9), 17, 0.65, (10,80) ] -> [200] 0.550453 (0.615713 / 0.497701)
- test02_12_2blur_all.npy : [ (9,9) 17, 0.6, (10,110) ] -> [200] 0.543039 (0.525870 / 0.561368)
- test02_13_2blur_all.npy : [ (9,9), 17, 0.6, (10,95) ] -> [200] 0.549547 (0.581407 / 0.520997)

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # 블러링
  gblur_img  = cv2.GaussianBlur(image, (11, 11), sigmaX = 0, sigmaY = 0)
  mblur_img  = cv2.medianBlur(image, ksize=15)

  # median blur + adaptive gaussian
  alpha = 0.45
  blend = cv2.addWeighted(gblur_img, 1-alpha, mblur_img, alpha, 0)

  # Apply Canny Edge Detection
  canny_edge = cv2.Canny(blend, threshold1=0, threshold2=110)

  #print(canny_edge)
  data_dic[selected_sample_name] = canny_edge



  # Display the original and adaptive thresholded images
  plt.figure(figsize=(15, 12))
  plt.subplot(1, 5, 1)
  plt.title('origin')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 2)
  plt.title('gaussian blur')
  plt.imshow(gblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 3)
  plt.title('median blur')
  plt.imshow(mblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 4)
  plt.title('blend(2+3)')
  plt.imshow(blend, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 5)
  plt.title('Canny')
  plt.imshow(canny_edge, cmap='gray')
  plt.axis('off')

  plt.show()

np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test02_14_2blur_all.npy', data_dic)


- test02_14_2blur_all.npy : [ (9,9), 19, 0.6, 0.6 (3,70) ] -> [200]  0.539588 (0.666622 / 0.453220)
- test02_15_2blur_all.npy : [ (9,9), 15, 0.3, 0.6, (3,70) ] -> [200] 0.541598 (0.583468 / 0.505335)
- test02_16_2blur_all.npy : [ (9,9), 17, 0.4, 0.6, (3,90) ] -> [200] 0.534752 (0.531394 / 0.538153)
- test02_17_2blur_all.npy : [ (9,9), 11, 0.6, 0.6 (3,70) ] -> [200] 0.485807 (0.753858 / 0.358378)
- test02_18_2blur_all.npy : [ (9,9), 11, 0.6, 0.6 (3,70) ] -> [200]

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #image = cv2.equalizeHist(image)

  # 블러링
  gblur_img  = cv2.GaussianBlur(image, (9, 9), sigmaX = 0, sigmaY = 0)
  mblur_img  = cv2.medianBlur(image, ksize=11)

  # median blur + adaptive gaussian
  alpha1 = 0.7
  blend = cv2.addWeighted(gblur_img, 1-alpha1, mblur_img, alpha1, 0)

  alpha2 = 0.7
  blend = cv2.addWeighted(blend, 1-alpha2, mblur_img, alpha2, 0)


  # Apply Canny Edge Detection
  canny_edge = cv2.Canny(blend, threshold1=30, threshold2=90)


  black = np.zeros_like(image)
  contours, _ = cv2.findContours(blend, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
  cv2.drawContours(black, contours, -1, (255, 0, 255), 2)



  #print(canny_edge)
  data_dic[selected_sample_name] = canny_edge



  # Display the original and adaptive thresholded images
  plt.figure(figsize=(15, 12))
  plt.subplot(1, 5, 1)
  plt.title('origin')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 2)
  plt.title('gaussian blur')
  plt.imshow(gblur_img, cmap='gray')
  plt.axis('off')

  plt.title('median blur')
  plt.imshow(mblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 4)
  plt.title('blend(2+3)')
  plt.imshow(blend, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 5)
  plt.title('contour')
  plt.imshow(black, cmap='gray')
  plt.axis('off')

  plt.show()

#np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test02_18_2blur_all.npy', data_dic)


# ⭐️bilateral filter
blend ( mblur + bilater )

- filter size -1 (자동값)으로 하면 원하는 대로 안 나옴. 구림

https://wjh2307.tistory.com/11


In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # bilateral
  bi = cv2.bilateralFilter(image, 80, 30, 30)

  # Display the original and adaptive thresholded images
  plt.figure(figsize=(12, 9))
  plt.subplot(1, 2, 1)
  plt.title('origin')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 2, 2)
  plt.title('bilateral')
  plt.imshow(bi, cmap='gray')
  plt.axis('off')

  plt.show()
  #np.save('/content/gdrive/MyDrive/CV/termProject/predictions/????????????.npy', data_dic)



#⭐️bilateral + mblur
생각보다 많이 구림. 뭔가 전반적으로 블러처리가 되어버려서 더 예민해진느낌..? 안잡아도 될 노이즈까지 잡힌다 자꾸 ㅜㅜ

미디안필터사이즈를 좀 줄이고... 미디안 비율을 높이면? bi의 엣지만 조금 추가해주는 느낌으로



```
[ mblur kernelsize, bilateral hyperparam, (mblur의)blend 비율, canny threshold ]
# of test image : [:20] -> F1 (Recall / Precision)
Overall Evaluation Metrics Using a Single Best Threshold for All Images

```



- test_bimblur01.npy : [ 7, (80,30,30), 0.9, (0,253) ] -> 0.531098 (0.608417 / 0.471215)     : 눈으로 보면 완전 너무 예민함. 근데 뭐야 둘 다 안 높고 걍 애매하네..
- test_bimblur06.npy : [9, (80,30,30), 0.9, (60,253) ] -> [200] 0.508616 (0.448007 / 0.588190)


In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # 블러링
  gblur_img  = cv2.GaussianBlur(image, (11, 11), sigmaX = 0, sigmaY = 0)
  mblur_img  = cv2.medianBlur(image, ksize=9)

  # bilateral
  bi = cv2.bilateralFilter(image, 80, 30, 30)


  # median blur + adaptive gaussian
  alpha = 0.9
  blend = cv2.addWeighted(bi, 1-alpha, mblur_img, alpha, 0)

  # Apply Canny Edge Detection
  canny_edge = cv2.Canny(blend, threshold1=60, threshold2=253)

  data_dic[selected_sample_name] = canny_edge


  # Display the original and adaptive thresholded images
  plt.figure(figsize=(15, 12))
  plt.subplot(1, 5, 1)
  plt.title('origin')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 2)
  plt.title('bilateral')
  plt.imshow(bi, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 3)
  plt.title('median blur')
  plt.imshow(mblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 4)
  plt.title('blend(2+3)')
  plt.imshow(blend, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 5)
  plt.title('Canny')
  plt.imshow(canny_edge, cmap='gray')
  plt.axis('off')

  plt.show()


  np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_bimblur06.npy', data_dic)



# bilateral -> mblur -> ...

bilateral을 해서, 엣지는 살리고 나머진 블러처리로 초벌을 한다.

초벌 사진에다가 median blur를 취하면, 애매한 엣지들만 사라지는 것을 기대..

```
[ (bilateral hyperparam), mblur kernelsize, (canny threshold) ]
# of test image : [:100] -> F1 (Recall / Precision)
Overall Evaluation Metrics Using a Single Best Threshold for All Images

```

- test_bimblur02.npy : [ (80,30,30), 7, (10,165) ] -> 0.499775 (0.532385 / 0.470930) 하 대체 뭐 어떻게 하라는 거 / 너무 디테일한 불필요한 애들도 잡힌다
- test_bimblur03.npy : [ (80,30,10), 11, (10,230) ] -> 0.465275 (0.376154 / 0.609736) : 아 recall이 엄청 낮게 잡혔네.....   

  [200] ->  0.477489 (0.395731 / 0.601826)
- test_bimblur04.npy : [ (80,30,10), 7, (30,150) ] ->  0.535038   0.502438   0.518226  : recall을 올리기 위해 포착하는 엣지를 늘려보았음

  [200] 0.517407 (0.556622 / 0.483353)

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # 블러링
  gblur_img  = cv2.GaussianBlur(image, (11, 11), sigmaX = 0, sigmaY = 0)


  # bilateral####
  bi = cv2.bilateralFilter(image, 80, 30, 10)
  mblur_img  = cv2.medianBlur(bi, ksize=7)
  ####

  # median blur + adaptive gaussian
  alpha = 0.9
  blend = cv2.addWeighted(bi, 1-alpha, mblur_img, alpha, 0)

  # Apply Canny Edge Detection
  canny_edge = cv2.Canny(mblur_img, threshold1=30, threshold2=150)
  ####

  data_dic[selected_sample_name] = canny_edge


  # Display the original and adaptive thresholded images
  plt.figure(figsize=(15, 12))
  plt.subplot(1, 5, 1)
  plt.title('origin')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 2)
  plt.title('bilateral')
  plt.imshow(bi, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 3)
  plt.title('bi->mblur')
  plt.imshow(mblur_img, cmap='gray')
  plt.axis('off')

  # plt.subplot(1, 5, 4)
  # plt.title('blend(2+3)')
  # plt.imshow(blend, cmap='gray')
  # plt.axis('off')

  plt.subplot(1, 5, 4)
  plt.title('Canny')
  plt.imshow(canny_edge, cmap='gray')
  plt.axis('off')

  plt.show()


  np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_bimblur04.npy', data_dic)



# mblur -> bilateral -> ...
mblur로 자잘한 노이즈를 과감하게 없앤 다음에 bilateral로 엣지를 살리는 방법... ㅈㅂ

```
[ mblur kernelsize, (bilateral hyperparam), (canny threshold) ]
# of test image : [:100] -> F1 (Recall / Precision)
Overall Evaluation Metrics Using a Single Best Threshold for All Images

```
- test_bimblur05.npy [ 11, (80,30,10),(10,100) ] : 0.512931
 (0.452920 / 0.591275)   
- mblur랑 블랜드를 해서.. bi 때문에 흐려진 엣지를 살리고,, canny를 빡세게 주면 ...


In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # 블러링
  gblur_img  = cv2.GaussianBlur(image, (11, 11), sigmaX = 0, sigmaY = 0)
  mblur_img  = cv2.medianBlur(image, ksize=11)


  # bilateral####
  bi = cv2.bilateralFilter(mblur_img, 80, 30, 10)
  #mblur_img  = cv2.medianBlur(bi, ksize=3)
  ####

  # median blur + adaptive gaussian
  alpha = 0.7
  blend = cv2.addWeighted(gblur_img, 1-alpha, mblur_img, alpha, 0)

  # Apply Canny Edge Detection
  canny_edge = cv2.Canny(bi, threshold1=10, threshold2=100)
  ####

  data_dic[selected_sample_name] = canny_edge


  # Display the original and adaptive thresholded images
  plt.figure(figsize=(15, 12))
  plt.subplot(1, 5, 1)
  plt.title('ori')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 2)
  plt.title('mblur')
  plt.imshow(mblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 3)
  plt.title('mblur->bi')
  plt.imshow(bi, cmap='gray')
  plt.axis('off')

  # plt.subplot(1, 5, 4)
  # plt.title('gb->bi->mb')
  # plt.imshow(mblur_img, cmap='gray')
  # plt.axis('off')

  plt.subplot(1, 5, 4)
  plt.title('Canny')
  plt.imshow(canny_edge, cmap='gray')
  plt.axis('off')

  plt.show()


  np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_bimblur05.npy', data_dic)



# gblur -> bilateral -> ...
gaussian blur를 통해서 전체적으로 흐릿하게 만든다.

그 후 bilateral을 통해서 그래도~!! 살아있는 엣지는 강조해준다. 를 기대. (근데 강도가 비슷한 엣지들이 사라진다 ㅜㅜ)

```
[ (gblur size), (bilateral hyperparam), mblur kernelsize, mblur의blend비율, (canny threshold) ]
# of test image : [:100] -> F1 (Recall / Precision)
Overall Evaluation Metrics Using a Single Best Threshold for All Images

```

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # 블러링
  gblur_img  = cv2.GaussianBlur(image, (11, 11), sigmaX = 0, sigmaY = 0)


  # bilateral####
  bi = cv2.bilateralFilter(gblur_img, 80, 30, 10)
  mblur_img  = cv2.medianBlur(bi, ksize=3)
  ####

  # median blur + adaptive gaussian
  alpha = 0.7
  blend = cv2.addWeighted(gblur_img, 1-alpha, mblur_img, alpha, 0)

  # Apply Canny Edge Detection
  canny_edge = cv2.Canny(blend, threshold1=10, threshold2=100)
  ####

  data_dic[selected_sample_name] = canny_edge


  # Display the original and adaptive thresholded images
  plt.figure(figsize=(15, 12))
  plt.subplot(1, 5, 1)
  plt.title('2+4 blend')
  plt.imshow(blend, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 2)
  plt.title('gblur')
  plt.imshow(gblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 3)
  plt.title('gblur->bi')
  plt.imshow(bi, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 4)
  plt.title('gb->bi->mb')
  plt.imshow(mblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 5)
  plt.title('Canny')
  plt.imshow(canny_edge, cmap='gray')
  plt.axis('off')

  plt.show()


  #np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_bigblur01.npy', data_dic)



# high-pass

```
[ radius ]
Threshold  Recall     Precision  F1-Score  
```
- test_highpass01.npy [ 20 ] : 0.047619 / 0.499239 / 0.163614 / 0.246457

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image_np = np.array(image)
  image = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

  # Apply Fourier Transform to the original image
  f_transform = np.fft.fft2(image)
  f_shift = np.fft.fftshift(f_transform)  # Shift the zero frequency component to the center

  # Calculate the magnitude spectrum
  magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)

  # Create a low-pass filter (circular mask)
  rows, cols = image.shape
  crow, ccol = rows // 2, cols // 2

  # Create a circular mask centered at the middle of the frequency domain
  mask = np.zeros((rows, cols), np.uint8)
  Y, X = np.ogrid[:rows, :cols]
  distance_from_center = np.sqrt((X - ccol)**2 + (Y - crow)**2)

  # Create a high-pass filter (circular mask)
  mask = np.ones((rows, cols), np.uint8)
  radius = 30
  mask[distance_from_center <= radius] = 0

  # Apply the mask to the shifted frequency representation
  f_high_pass = f_shift * mask

  # Calculate the magnitude spectrum of the filtered frequency
  magnitude_spectrum_high_pass = 20 * np.log(np.abs(f_high_pass) + 1)

  # Inverse Fourier Transform to get the filtered image
  f_ishift = np.fft.ifftshift(f_high_pass)
  high_pass_img = np.fft.ifft2(f_ishift)
  high_pass_img = np.abs(high_pass_img)

  data_dic[selected_sample_name] = high_pass_img

  # Display the original image, filter, magnitude spectrum, and filtered image side by side
  plt.figure(figsize=(24, 6))

  plt.subplot(1, 4, 1)
  plt.title('Original Grayscale Image')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 4, 2)
  plt.title('High-pass Filter (Circular Mask)')
  plt.imshow(mask, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 4, 3)
  plt.title('Magnitude Spectrum After High-pass Filtering')
  plt.imshow(magnitude_spectrum_high_pass, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 4, 4)
  plt.title('High-pass Filtered Image')
  plt.imshow(high_pass_img, cmap='gray')
  plt.axis('off')

  plt.show()


  #np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_highpass01.npy', data_dic)

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

    image = dataset.read_image(selected_sample_name)
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)  # normalize
    image_np = np.array(image)
    image = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

    # Apply Fourier Transform to the original image
    f_transform = np.fft.fft2(image)
    f_shift = np.fft.fftshift(f_transform)  # Shift the zero frequency component to the center

    # Calculate the magnitude spectrum
    magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)

    # Create a low-pass filter (circular mask)
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    # Create a circular mask centered at the middle of the frequency domain
    mask = np.zeros((rows, cols), np.uint8)
    Y, X = np.ogrid[:rows, :cols]
    distance_from_center = np.sqrt((X - ccol) ** 2 + (Y - crow) ** 2)

    # Create a high-pass filter (circular mask)
    mask = np.ones((rows, cols), np.uint8)
    radius = 30
    mask[distance_from_center <= radius] = 0

    # Apply the mask to the shifted frequency representation
    f_high_pass = f_shift * mask

    # Calculate the magnitude spectrum of the filtered frequency
    magnitude_spectrum_high_pass = 20 * np.log(np.abs(f_high_pass) + 1)

    # Inverse Fourier Transform to get the filtered image
    f_ishift = np.fft.ifftshift(f_high_pass)
    high_pass_img = np.fft.ifft2(f_ishift)
    high_pass_img = np.abs(high_pass_img)

    # Convert high-pass image to uint8
    high_pass_img_uint8 = np.uint8(high_pass_img)

    # Apply adaptive thresholding
    adaptive_thresh_mean = cv2.adaptiveThreshold(
        high_pass_img_uint8, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 101, 17)

    # Black image for drawing contours
    black = np.zeros(image.shape, np.uint8)

    # Find and draw contours
    contours, _ = cv2.findContours(adaptive_thresh_mean, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(black, contours, -1, 255, 3)

    # Save result to dictionary
    data_dic[selected_sample_name] = black

    # Display results
    plt.figure(figsize=(24, 6))

    plt.subplot(1, 4, 1)
    plt.title('Original Grayscale Image')
    plt.imshow(image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.title('High-pass Filter (Circular Mask)')
    plt.imshow(mask, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.title('Magnitude Spectrum After High-pass Filtering')
    plt.imshow(magnitude_spectrum_high_pass, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.title('Black')
    plt.imshow(black, cmap='gray')
    plt.axis('off')

    plt.show()

# Save data dictionary if needed
# np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_highpass01.npy', data_dic)

# high pass 결과에다가 median 적용?ㅋㅋ

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image_np = np.array(image)
  image = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

  # Apply Fourier Transform to the original image
  f_transform = np.fft.fft2(image)
  f_shift = np.fft.fftshift(f_transform)  # Shift the zero frequency component to the center

  # Calculate the magnitude spectrum
  magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)

  # Create a low-pass filter (circular mask)
  rows, cols = image.shape
  crow, ccol = rows // 2, cols // 2

  # Create a circular mask centered at the middle of the frequency domain
  mask = np.zeros((rows, cols), np.uint8)
  Y, X = np.ogrid[:rows, :cols]
  distance_from_center = np.sqrt((X - ccol)**2 + (Y - crow)**2)

  # Create a high-pass filter (circular mask)
  mask = np.ones((rows, cols), np.uint8)
  radius = 20
  mask[distance_from_center <= radius] = 0

  # Apply the mask to the shifted frequency representation
  f_high_pass = f_shift * mask

  # Calculate the magnitude spectrum of the filtered frequency
  magnitude_spectrum_high_pass = 20 * np.log(np.abs(f_high_pass) + 1)

  # Inverse Fourier Transform to get the filtered image
  f_ishift = np.fft.ifftshift(f_high_pass)
  high_pass_img = np.fft.ifft2(f_ishift)
  high_pass_img = np.abs(high_pass_img)

  high_pass_img = cv2.normalize(high_pass_img, None, 0, 255, cv2.NORM_MINMAX)
  final = cv2.medianBlur(high_pass_img, ksize=3)

  data_dic[selected_sample_name] = final

  # Display the original image, filter, magnitude spectrum, and filtered image side by side
  plt.figure(figsize=(24, 6))

  plt.subplot(1, 4, 1)
  plt.title('Original Grayscale Image')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 4, 2)
  plt.title('High-pass Filter')
  plt.imshow(final, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 4, 3)
  plt.title('fiinal')
  plt.imshow(high_pass_img, cmap='gray')
  plt.axis('off')

  # plt.subplot(1, 4, 4)
  # plt.title('final')
  # plt.imshow(final, cmap='gray')
  # plt.axis('off')

  plt.show()


  #np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_highpass01.npy', data_dic)

# mblur + highpass filter

mblur로 두꺼운 엣지만 남기고, highpass filter로 형태만 추출하고픈 마음
개ㅐㅐㅐㅐㅐㅐㅐㅐㅐㅐ구림

```
[ mblurkernelsize , radius ]
```

- test_mblurhigh01.npy [

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image_np = np.array(image)
  image = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

  mblur_img  = cv2.medianBlur(image, ksize=9)

  # Apply Fourier Transform to the original image
  f_transform = np.fft.fft2(mblur_img)
  f_shift = np.fft.fftshift(f_transform)  # Shift the zero frequency component to the center

  # Calculate the magnitude spectrum
  magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)

  # Create a low-pass filter (circular mask)
  rows, cols = mblur_img.shape
  crow, ccol = rows // 2, cols // 2

  # Create a circular mask centered at the middle of the frequency domain
  mask = np.zeros((rows, cols), np.uint8)
  Y, X = np.ogrid[:rows, :cols]
  distance_from_center = np.sqrt((X - ccol)**2 + (Y - crow)**2)

  # Create a high-pass filter (circular mask)
  mask = np.ones((rows, cols), np.uint8)
  radius = 30
  mask[distance_from_center <= radius] = 0

  # Apply the mask to the shifted frequency representation
  f_high_pass = f_shift * mask

  # Calculate the magnitude spectrum of the filtered frequency
  #magnitude_spectrum_high_pass = 20 * np.log(np.abs(f_high_pass) + 1)

  # Inverse Fourier Transform to get the filtered image
  f_ishift = np.fft.ifftshift(f_high_pass)
  high_pass_img = np.fft.ifft2(f_ishift)
  high_pass_img = np.abs(high_pass_img)

  data_dic[selected_sample_name] = high_pass_img

  # Display the original image, filter, magnitude spectrum, and filtered image side by side
  plt.figure(figsize=(24, 6))

  plt.subplot(1, 4, 1)
  plt.title('Original Grayscale Image')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 4, 2)
  plt.title('median blur')
  plt.imshow(mblur_img, cmap='gray')
  plt.axis('off')

  # plt.subplot(1, 4, 3)
  # plt.title('Magnitude Spectrum After High-pass Filtering')
  # plt.imshow(magnitude_spectrum_high_pass, cmap='gray')
  # plt.axis('off')

  plt.subplot(1, 4, 3)
  plt.title('High-pass Filtered Image')
  plt.imshow(high_pass_img, cmap='gray')
  plt.axis('off')

  plt.show()


  np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_mblurhigh01.npy', data_dic)

# graph cut

In [ ]:
from skimage import data, color, graph
from skimage.color import rgb2lab, label2rgb
from skimage.segmentation import slic, mark_boundaries

data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  gimage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # 블러링
  gblur_img  = cv2.GaussianBlur(image, (3, 3), sigmaX = 0, sigmaY = 0)
  mblur_img  = cv2.medianBlur(gimage, ksize=19)
  # median blur + adaptive gaussian
  alpha1 = 0.5
  #blur_blend = cv2.addWeighted(gblur_img, 1-alpha1, mblur_img, alpha1, 0)


###############
  # Apply SLIC SuperPixel segmentation -> 픽셀 개수를 좀 줄여주는 역할,,,,
  segments = slic(gblur_img, n_segments=100, compactness=20, sigma=0.8)
  # compactness가 커지면 굵직굵직해짐

  # Build Region Adjacency Graph (RAG)
  rag = graph.rag_mean_color(image, segments, mode='similarity')

  # Apply Normalized Cut
  labels = graph.cut_normalized(segments, rag)
  print("# Unique Labels:", len(np.unique(labels)))

  segmented_image = label2rgb(labels, image, kind='avg', bg_label=0) # rgb label로 줌
  seg_image = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)

  alpha2 = 0.2
  #blend = cv2.addWeighted(blur_blend, 1-alpha2, seg_image, alpha2, 0)


############
  # Apply Canny Edge Detection
  canny_edge = cv2.Canny(segmented_image, threshold1=70, threshold2=150)

  #print(canny_edge)
  data_dic[selected_sample_name] = canny_edge



  # Display the original and adaptive thresholded images
  # plt.figure(figsize=(18, 6))
  # plt.subplot(2, 6, 1)
  # plt.title('origin')
  # plt.imshow(image, cmap='gray')
  # plt.axis('off')

  # plt.subplot(2, 6, 2)
  # plt.title('gaussian blur')
  # plt.imshow(gblur_img, cmap='gray')
  # plt.axis('off')

  # plt.subplot(2, 6, 3)
  # plt.title('median blur')
  # plt.imshow(mblur_img, cmap='gray')
  # plt.axis('off')

  plt.subplot(1, 2, 1)
  plt.title('Graph Cut')
  plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
  plt.axis('off')

  # plt.subplot(2, 2, 1)
  # plt.title('blend(2+3+4)')
  # plt.imshow(blend, cmap='gray')
  # plt.axis('off')

  plt.subplot(1, 2, 2)
  plt.title('Canny on Graph cut')
  plt.imshow(canny_edge, cmap='gray')
  plt.axis('off')

  plt.show()


  #np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test_graph.npy', data_dic)

# 리더보드 코드

In [ ]:
def blend4(image, threshold1, threshold2) :
  g_image1 = cv2.GaussianBlur(image, (15, 15), 0)
  g_image2 = cv2.GaussianBlur(image, (29, 29), 0)

  m_image1 = cv2.medianBlur(image, 15)
  m_image2 = cv2.medianBlur(image, 29)

  alpha = 0.3
  blended1 = cv2.addWeighted(g_image1, alpha, m_image1, 1-alpha, 0, dst=None, dtype=None)

  alpha2 = 0.6
  blended2 = cv2.addWeighted(g_image2, alpha2, m_image2, 1-alpha2, 0, dst=None, dtype=None)

  alpha3 = 0.6
  blended3 = cv2.addWeighted(blended1, alpha3, blended2, 1-alpha3, 0, dst=None, dtype=None)
  edges3 = cv2.Canny(blended3, threshold1, threshold2, apertureSize=3)

  return blended3, edges3

In [ ]:
import sys

def preprocess_image(image):
    """Reshape the image to a 2D array of pixels and 3 color values (RGB) and convert to float."""
    pixel_values = image.reshape((-1, 3))
    return np.float32(pixel_values)

def perform_kmeans_clustering(pixel_values, k=3):
    """Perform k-means clustering on the pixel values."""
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    compactness, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    return compactness, labels, np.uint8(centers)

def create_segmented_image(pixel_values, labels, centers):
    """Create a segmented image using the cluster centroids."""
    segmented_image = centers[labels.flatten()]
    return segmented_image.reshape(image.shape)

def ensemble_edges(edge1, edge2, method='or', alpha=0.5, beta=0.5):
    if method == 'or':
        # 픽셀 단위 OR 연산 (하나라도 경계인 경우 유지)
        combined_edges = cv2.bitwise_or(edge1, edge2)
    elif method == 'and':
        # 픽셀 단위 AND 연산 (두 결과가 공통으로 경계인 경우만 유지)
        combined_edges = cv2.bitwise_and(edge1, edge2)
    elif method == 'weighted':
        # 가중치 블렌딩
        combined_edges = cv2.addWeighted(edge1, alpha, edge2, beta, 0)
    else:
        raise ValueError("method는 'or', 'and', 또는 'weighted' 중 하나여야 합니다.")

    return combined_edges


In [ ]:
for selected_sample_name in dataset.sample_names:
    #print(f"Processing image {num} : {split}/{selected_sample_name}")
    image = dataset.read_image(selected_sample_name)
    image_np = np.array(image)
    image = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    blended, edge = blend4(image, 30 ,50)
    pixel_values = preprocess_image(blended)

    k=3
    # compactness is the sum of squared distance from each point to their corresponding centers
    compactness, labels, centers = perform_kmeans_clustering(pixel_values, k)
    # create the segmented image
    segmented_image = create_segmented_image(pixel_values, labels, centers)

    edge2 = cv2.Canny(segmented_image, 50, 100, apertureSize=3)
    # 특정 클러스터의 경계 감지
    #edge2 = highlight_cluster_edges(segmented_image, labels, cluster_to_highlight, 130, 180)

    # 앙상블 적용
    combined_or = ensemble_edges(edge, edge2, method='or')
    combined_and = ensemble_edges(edge, edge2, method='and')
    combined_weighted = ensemble_edges(edge, edge2, method='weighted', alpha=0.6, beta=0.4)

    threshold_value = 152.5
    _, strong_edges = cv2.threshold(combined_weighted, threshold_value, 255, cv2.THRESH_BINARY)

    combined_weighted = ensemble_edges(edge, strong_edges, method='weighted', alpha=0.6, beta=0.4)

    # 결과 시각화
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 4, 1)
    plt.title("Original Image")
    plt.imshow(image, cmap="gray")
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.title("combined_weighted")
    plt.imshow(combined_weighted, cmap="gray")
    plt.axis("off")

    plt.show()

## Try 3

In [ ]:
data_dic = {}
for selected_sample_name in dataset.sample_names:

  image = dataset.read_image(selected_sample_name)
  image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX) #normalize
  image = np.array(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # 블러링
  gblur_img  = cv2.GaussianBlur(image, (11, 11), sigmaX = 0, sigmaY = 0)
  mblur_img  = cv2.medianBlur(image, ksize=15)

  # median blur + adaptive gaussian
  alpha = 0.45
  blend = cv2.addWeighted(gblur_img, 1-alpha, mblur_img, alpha, 0)

  # Apply Canny Edge Detection
  canny_edge = cv2.Canny(blend, threshold1=0, threshold2=110)

  #print(canny_edge)
  data_dic[selected_sample_name] = canny_edge



  # Display the original and adaptive thresholded images
  plt.figure(figsize=(15, 12))
  plt.subplot(1, 5, 1)
  plt.title('origin')
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 2)
  plt.title('gaussian blur')
  plt.imshow(gblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 3)
  plt.title('median blur')
  plt.imshow(mblur_img, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 4)
  plt.title('blend(2+3)')
  plt.imshow(blend, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 5, 5)
  plt.title('Canny')
  plt.imshow(canny_edge, cmap='gray')
  plt.axis('off')

  plt.show()

np.save('/content/gdrive/MyDrive/CV/termProject/predictions/test02_14_2blur_all.npy', data_dic)
